In [1]:
import cv2
import numpy as np
import dlib
import pandas as pd
import os, glob
from imutils import face_utils
# import numpy as np
import argparse
import imutils

In [2]:
# Load the detector
detector = dlib.get_frontal_face_detector()

In [3]:
# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
emotion_codes = {
    'Angry': 0,
    'Disgust' : 1, 
    'Fear' : 2, 
    'Happy' : 3, 
    'Sad' : 4, 
    'Surprise' : 5, 
    'Neutral': 6
   }

In [45]:
emotions_features = pd.DataFrame()
emotions_features2 = pd.DataFrame()

for filepath in glob.glob("photo_predict\\*.jpg"):
# for filepath in glob.glob("Training_803556.jpg"):
    #Split a path to get emotion name
    emotion = filepath.split(os.sep)[1]
#     print(emotion)
    
    # Match dictionary key format from folder name
    # emotion_code = emotion_codes[emotion.capitalize()]
    
    # Read the image
    img = cv2.imread(filepath)
    
    # Convert image into grayscale
    gray = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2GRAY)
#     print(gray)
    
    # Use detector to find landmarks
    faces = detector(gray)
    
    for face in faces:
        x1 = face.left() # left point
        y1 = face.top() # top point
        x2 = face.right() # right point
        y2 = face.bottom() # bottom point


        landmarks = predictor(image=gray, box=face)


        # Create empty arrays per face feature
        jaw = np.array([])
        left_eyebrow = np.array([])
        right_eyebrow = np.array([])
        nose = np.array([])
        left_eye = np.array([])
        right_eye = np.array([])
        mouth = np.array([])
        lips = np.array([])
        result = np.array([])
        emptyarray = []
        
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            # Draw a circle
            cv2.circle(img=img, center=(x, y), radius=3, color=(0, 255, 0), thickness=-1)
#             print(x)
            # Created a coordinate numpy array
            coordinates = np.array([x, y])

            result = np.hstack((result, coordinates))
            if n >= 0 and n <=16:
                jaw=np.hstack((jaw, coordinates))
#                 print(jaw)

            elif n > 16 and n <=21:
                right_eyebrow=np.hstack((right_eyebrow, coordinates))  

            elif n > 21 and n <=26:
                left_eyebrow=np.hstack((left_eyebrow, coordinates))

            elif n > 26 and n <=35:
                nose=np.hstack((nose, coordinates))

            elif n > 35 and n <=41:
                right_eye=np.hstack((right_eye, coordinates))

            elif n > 41 and n <=47:
                left_eye=np.hstack((left_eye, coordinates))

            elif n > 47 and n <=60:
                mouth=np.hstack((mouth, coordinates))

            elif n > 60 and n <=67:
                lips=np.hstack((lips, coordinates))
            emptyarray.append(coordinates[0])
            emptyarray.append(coordinates[1])

            df2 = pd.DataFrame({'features':[emptyarray]})
           
        
        # emotions_features = emotions_features.append(df)
        emotions_features2 = emotions_features2.append(df2)
        
# show the image
cv2.imshow(winname="Face", mat=img)
# Wait for a key press to exit
cv2.waitKey(delay=0)
# Close all windows
cv2.destroyAllWindows()

In [6]:
emotions_features2.head(1)

,features
0,"[709, 526, 718, 585, 730, 643, 747, 701, 776, ..."


In [7]:
emotions_features2['features']= emotions_features2['features'].astype(str).str[1:-1]

emotions_features3 = emotions_features2['features'].str.split(",", n = 135, expand = True) 
emotions_features3.rename(columns= lambda s:f"data{s}", inplace=True)


In [8]:
data_int = emotions_features3.apply(pd.to_numeric)
data_int.fillna(0,inplace=True)
data_int = data_int.astype({"data0": int})
data_int

,data0,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data126,data127,data128,data129,data130,data131,data132,data133,data134,data135
0,709,526,718,585,730,643,747,701,776,753,...,996,697,1056,693,998,730,970,736,940,735


 ## Load the Model

In [9]:
from tensorflow.keras.models import load_model
model =load_model("image_emotions_model.h5")

## Make the Prediction

In [39]:
prediction = model.predict_classes(data_int)
prediction[0]

3

In [40]:
result_df = pd.DataFrame()
result_df['Predicted'] = encoded_predictions
result_df = result_df.astype({"Predicted": int})
result_df.dtypes

Predicted    int32
dtype: object

In [41]:
emotion_codes = {
    0: 'Angry',
    1:'Disgust' ,
    2:'Fear',
    3:'Happy',
    4:'Sad',
    5:'Surprise',
    6:'Neutral'
   }

In [43]:
result = emotion_codes[prediction[0]]
result

'Happy'